# Regressão Linear Múltipla com NumPy

In [8]:
import numpy as np
import math
import time

### Versão Não Vetorizada
Função para calcular o MSE (Mean Squared Error):

$MSE(\hat{w}) = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i (x_i))^2$

In [2]:
# y = mx + b
# m is slope, b is y-intercept
def calculate_hypothesis(b, m, X):
    h = b
    for i in range(m.size):
        h += X[i] * m[i]
    return h


def compute_mse(b, m, points):
    totalError = 0
    for i in range(0, len(points)):
        X = points[i, :-1]
        y = points[i, -1]
        totalError += (y - calculate_hypothesis(b, m, X)) ** 2
    return totalError / float(len(points))

Função para fazer uma atualização dos parâmetros no Gradiente Descendente:

$w_0 = w_0 + 2\alpha\sum_{i=1}^N (y_i - (w_0+w_1x_{i_1}+w_2x_{i_2}+\ldots+w_nx_{i_n}))$

$w_1 = w_1 + 2\alpha\sum_{i=1}^N x_{i_1}(y_i - (w_0+w_1x_{i_1}+w_2x_{i_2}+\ldots+w_nx_{i_n}))$

...

$w_n = w_n + 2\alpha\sum_{i=1}^N x_{i_n}(y_i - (w_0+w_1x_{i_1}+w_2x_{i_2}+\ldots+w_nx_{i_n}))$


In [3]:
def step_gradient(b_current, m_current, points, learningRate):
    b_gradient = 0
    m_gradient = np.zeros(m_current.size)
    for i in range(0, len(points)):
        X = points[i, :-1]
        y = points[i, -1]
        
        h = calculate_hypothesis(b_current, m_current, X)
        b_gradient += y - h
        
        for j in range(m_gradient.size):
            m_gradient[j] += X[j] * (y - h)
            
    new_b = b_current + (2 * learningRate * b_gradient)
    new_m = np.add(m_current, 2 * learningRate * m_gradient)
    
    return [new_b, new_m, b_gradient, m_gradient]

$||\mathbf{w}||_2 = \sqrt{\sum_{j=1}^D w_j^2}$

In [4]:
def norm_2(x):
    c=0
    for i in range(len(x)):
        c += x[i]**2
    return math.sqrt(c)

Função para iterar sobre o gradiente descendente até convergência.

In [5]:
def gradient_descent_runner(points, starting_b, starting_m, learning_rate, epsilon):
    b = starting_b
    m = starting_m
    grad = np.array([np.inf] * (starting_m.size + 1))
    i = 0
    while (norm_2(grad)>=epsilon):
        b, m, b_gradient, m_gradient = step_gradient(b, m, points, learning_rate)
        grad = [b_gradient]
        grad.extend(m_gradient)
        grad = np.array(grad)
        if i % 1000 == 0:
            #print(norm_2(grad))
            print("MSE na iteração {0} é de {1}".format(i,compute_mse(b,m,points)))
        i+= 1
    return [b, m]

In [10]:
points = np.genfromtxt("sample_treino.csv", delimiter=",")
learning_rate = 0.00003
initial_b = 0 # initial y-intercept guess
initial_m = np.zeros(points[0].size - 1) # initial slope guess
#num_iterations = 10000
epsilon = 0.5
print("Starting gradient descent at b = {0}, m = {1}, error = {2}".format(initial_b, initial_m, compute_mse(initial_b, initial_m, points)))
print("Running...")
tic = time.time()
[b, m] = gradient_descent_runner(points, initial_b, initial_m, learning_rate, epsilon)
toc = time.time()
print("Gradiente descendente convergiu com w0 = {0}, w1 = {1}, erro = {2}".format(b, m, compute_mse(b, m, points)))
print("Versão não-vetorizada rodou em: " + str(1000*(toc-tic)) + "ms")

Starting gradient descent at b = 0, m = [0. 0. 0. 0. 0.], error = 54.4799538556
Running...
MSE na iteração 0 é de 15.3941521144
MSE na iteração 1000 é de 0.430362693446
MSE na iteração 2000 é de 0.428912823578
MSE na iteração 3000 é de 0.427666789365
MSE na iteração 4000 é de 0.426509325361
MSE na iteração 5000 é de 0.425433906534
MSE na iteração 6000 é de 0.424434716667
MSE na iteração 7000 é de 0.423506352419
MSE na iteração 8000 é de 0.422643793454
MSE na iteração 9000 é de 0.421842375293
MSE na iteração 10000 é de 0.42109776409
MSE na iteração 11000 é de 0.420405933195
MSE na iteração 12000 é de 0.419763141379
MSE na iteração 13000 é de 0.419165912603
MSE na iteração 14000 é de 0.41861101722
MSE na iteração 15000 é de 0.418095454511
MSE na iteração 16000 é de 0.417616436455
MSE na iteração 17000 é de 0.417171372657
MSE na iteração 18000 é de 0.416757856335
MSE na iteração 19000 é de 0.416373651308
MSE na iteração 20000 é de 0.416016679903
Gradiente descendente convergiu com w0 = 0.

### Versão Vetorizada

$MSE(\hat{w})=\frac{1}{N}(y-\hat{\mathbf{w}}^T\mathbf{x})^T(y-\hat{\mathbf{w}}^T\mathbf{x})$

In [4]:
def compute_mse_vectorized(w,X,Y):
    res = Y - np.dot(X,w)
    totalError = np.dot(res.T,res)
    return totalError / float(len(Y))

In [5]:
def step_gradient_vectorized(w_current,X,Y,learningRate):
    res = Y - np.dot(X,w_current)
    gradient = np.matmul(X.T, res)
    new_w = np.add(w_current, 2 * learningRate * gradient)
    return [new_w, gradient]

In [6]:
def gradient_descent_runner_vectorized(starting_w, X,Y, learning_rate, epsilon):
    w = starting_w
    grad = np.array([np.inf] * starting_w.size)
    i = 0
    while (np.linalg.norm(grad)>=epsilon):
        [w, grad] = step_gradient_vectorized(w, X, Y, learning_rate)
        #print(np.linalg.norm(grad))
        if i % 1000 == 0:
            print("MSE na iteração {0} é de {1}".format(i,compute_mse_vectorized(w, X, Y)))
        i+= 1
    return w

In [9]:
points = np.genfromtxt("sample_treino.csv", delimiter=",")
points = np.c_[np.ones(len(points)),points]
X = points[:, :-1]
Y = points[:, -1]
init_w = np.zeros(X[0].size)
learning_rate = 0.00003
#num_iterations = 10000
epsilon = 0.5
print("Starting gradient descent at w0 = {0}, w1 = {1}, error = {2}".format(init_w[0], init_w[1:], compute_mse_vectorized(init_w, X,Y)))
print("Running...")
tic = time.time()
w = gradient_descent_runner_vectorized(init_w, X,Y, learning_rate, epsilon)
toc = time.time()
print("Gradiente descendente convergiu com w0 = {0}, w1 = {1}, error = {2}".format(w[0], w[1:], compute_mse_vectorized(w,X,Y)))
print("Versão vetorizada rodou em: " + str(1000*(toc-tic)) + " ms")


Starting gradient descent at w0 = 0.0, w1 = [0. 0. 0. 0. 0.], error = 54.4799538556
Running...
MSE na iteração 0 é de 15.3941521144
MSE na iteração 1000 é de 0.430362693446
MSE na iteração 2000 é de 0.428912823578
MSE na iteração 3000 é de 0.427666789365
MSE na iteração 4000 é de 0.426509325361
MSE na iteração 5000 é de 0.425433906534
MSE na iteração 6000 é de 0.424434716667
MSE na iteração 7000 é de 0.423506352419
MSE na iteração 8000 é de 0.422643793454
MSE na iteração 9000 é de 0.421842375293
MSE na iteração 10000 é de 0.42109776409
MSE na iteração 11000 é de 0.420405933195
MSE na iteração 12000 é de 0.419763141379
MSE na iteração 13000 é de 0.419165912603
MSE na iteração 14000 é de 0.41861101722
MSE na iteração 15000 é de 0.418095454511
MSE na iteração 16000 é de 0.417616436455
MSE na iteração 17000 é de 0.417171372657
MSE na iteração 18000 é de 0.416757856335
MSE na iteração 19000 é de 0.416373651308
MSE na iteração 20000 é de 0.416016679903
Gradiente descendente convergiu com w0 

### Comparando com o SciKit Learn

In [40]:
import sklearn.linear_model


# carregar modelo e treiná-lo
r = sklearn.linear_model.LinearRegression()
r.fit(X, Y)

print "coeficientes do SciKit: %s" % r.coef_[1:]
print "coeficientes deste lab: %s" % w[1:]



coeficientes do SciKit: [0.10304143 0.0464367  0.16409834 0.38117843 0.02027816]
coeficientes deste lab: [0.11762689 0.08368747 0.16224371 0.42251182 0.03029501]


Mesmo com uma certa divergência (mínima, em certos coeficientes), os resultados obtidos pelo modelo deste lab são parecidos com os coeficientes mostrados com a biblioteca SciLearn Kit.